In [2]:
from moviepy import VideoFileClip, AudioFileClip

# Load the video and audio files
video = VideoFileClip("cat_in_sunglasses.mp4")
audio = AudioFileClip("alexander.mp3")

# Get the duration of the video
video_duration = video.duration

# Adjust the audio duration to match the video duration
if audio.duration < video_duration:
    # If the audio is shorter, loop it to match the video's duration
    audio = audio.fx(afx.audio_loop, duration=video_duration)
elif audio.duration > video_duration:
    # If the audio is longer, trim it to the video's duration
    audio = audio.with_duration(video_duration)


# Set the new audio to the video
final_video = video.with_audio(audio)

# Export the final video
final_video.write_videofile("output_with_music.mp4", codec="libx264", audio_codec="aac")


MoviePy - Building video output_with_music.mp4.
MoviePy - Writing audio in output_with_musicTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video output_with_music.mp4



MoviePy - Done !
MoviePy - video ready output_with_music.mp4


In [9]:
import os
import pickle
import time
import httplib2
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, HttpRequest
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

import googleapiclient.http

# -----------------------------
# CONFIG
# -----------------------------
VIDEO_FILE = "generate_a_fun_and_engaging_youtube_video_title_fo_20250724_131554_with_audio.mp4"
VIDEO_TITLE = "Cute Animal Heroes | Fun Kids Animation Ideas!"
VIDEO_DESCRIPTION = "Discover 10 fun and realistic cartoon-style animal characters like a lion knight and owl librarian, brought to life in magical settings—from cheese moons to jelly oceans! Perfect for kids’ animations with watercolor, 3D, or cartoon styles. Get inspired with imaginative actions and whimsical worlds!"
VIDEO_TAGS = "#KidsAnimation #CartoonAnimals #CreativeCharacters #AnimalCartoons #FunForKids #3DAnimation #WatercolorArt #MagicalWorlds #CuteCartoons #AnimationIdeas"
VIDEO_CATEGORY_ID = "22"  # People & Blogs
VIDEO_PRIVACY = "public"
CREDENTIALS_FILE = "client_secrets.json"
TOKEN_FILE = "token.pkl"

# -----------------------------
# AUTHENTICATION
# -----------------------------
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]

def authenticate_youtube():
    credentials = None

    if os.path.exists(TOKEN_FILE):
        with open(TOKEN_FILE, "rb") as f:
            credentials = pickle.load(f)

    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            system = platform.system().lower()
            if system == "darwin":  # macOS
                credentials = flow.run_local_server(port=0)
            else:  # Linux, Windows, etc.
                credentials = flow.run_console()
        with open(TOKEN_FILE, "wb") as f:
            pickle.dump(credentials, f)

    return build("youtube", "v3", credentials=credentials)

# -----------------------------
# VIDEO UPLOAD
# -----------------------------
def upload_video(youtube):
    media = MediaFileUpload(VIDEO_FILE, mimetype="video/mp4", resumable=True)

    request = youtube.videos().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": VIDEO_TITLE,
                "description": VIDEO_DESCRIPTION,
                "categoryId": VIDEO_CATEGORY_ID,
                "tags" : VIDEO_TAGS
            },
            "status": {
                "privacyStatus": VIDEO_PRIVACY,
            },
        },
        media_body=media
    )

    response = None
    while response is None:
        try:
            status, response = request.next_chunk()
            if status:
                print(f"Uploading: {int(status.progress() * 100)}%")
        except Exception as e:
            print(f"❌ Error during upload: {e}")
            break

    if response:
        print(f"✅ Uploaded: https://youtu.be/{response['id']}")



# -----------------------------
# RUN
# -----------------------------
if __name__ == "__main__":
    youtube = authenticate_youtube()
    print("authentication done")
    upload_video(youtube)


authentication done
✅ Uploaded: https://youtu.be/AltFDgdnFJk
